In [307]:
"""
Scrape Ghana Districts data from https://www.citypopulation.de/
"""
import bs4
import requests
from lxml import html
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(rc={'figure.figsize':(20,16)})
#ns.set_theme(style='darkgrid')

In [308]:
#get web data

#link for web data
url="https://www.citypopulation.de/en/ghana/admin/"

res=requests.get(url)
res.raise_for_status() #check if link is valid

<h1><b>BEAUTIFULSOUP</b></h1>

In [309]:
#parse web data using bs4

data=bs4.BeautifulSoup(res.content, 'html.parser')

In [310]:
# get table columns which should be marked with table head tag 'th'

head = data.find_all('th')

columns=[]

for td in head:
    columns.append(td.text)

columns=columns[:4]
#print(columns)

In [319]:
body = data.find_all('tbody')


body_rows=[]

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
body_rows = body_rows[:len(body_rows)-4]
#print(body_rows)

In [312]:
regions = []
i = 0
while i < len(body_rows):
    regions.append(body_rows[i])
    i+=2

#print(regions) 

In [313]:
regcolumns=columns.copy()
regcolumns.append('Misc')

In [314]:
df = pd.DataFrame(regions, columns=regcolumns)
df = df.drop(df[['Misc', 'Status']],axis=1)
df.columns = ['Region', 'Census', 'Projected']
df

,Region,Census,Projected
0,Ahafo (← Brong Ahafo),"484,210","613,049"
1,Ashanti,"4,780,380","5,924,498"
2,Bono (← Brong Ahafo),"922,617","1,168,807"
3,Bono East (← Brong Ahafo),"904,156","1,133,768"
4,Central,"2,201,863","2,605,492"
5,Eastern,"2,633,154","3,318,853"
6,Greater Accra,"4,010,054","5,055,883"
7,North East (← Northern),"465,005","588,800"
8,Northern,"1,544,946","1,948,913"
9,Oti (← Volta),"605,725","759,799"


In [325]:
district_row = []

i = 1
while i < len(body_rows):
    district_row.append(body_rows[i])
    i+=2
print(len(district_row))

16


In [331]:
district = []
status = []
census = []
projected = []

In [332]:
for row in district_row:
    j=0
    while j < len(row):
        district.append(row[j])
        j+=5
#print(len(district))

260


In [333]:
for row in district_row:
    j=1
    while j < len(row):
        status.append(row[j])
        j+=5
#print(len(status))

260


In [334]:
for row in district_row:
    j=2
    while j < len(row):
        census.append(row[j])
        j+=5
#print(len(census))

260


In [335]:
for row in district_row:
    j=3
    while j < len(row):
        projected.append(row[j]) 
        j+=5
#print(len(projected))

260


In [336]:
df=pd.DataFrame(data={'District':district, 'Status':status, 'Census':census, 'Projected':projected})
df.replace(',','',regex=True,inplace=True)
df[['Census', 'Projected']]=dist_pop[['Census','Projected']].astype('int64')
df

,District,Status,Census,Projected
0,Asunafo North Municipal,Municipal District,124685,157870
1,Asunafo South,District,95580,120976
2,Asutifi North,District,52259,66141
3,Asutifi South,District,53584,67763
4,Tano North Municipal,Municipal District,79973,101305
...,...,...,...,...
255,Bodi,District,53314,71133
256,Juabeso,District,58435,77963
257,Sefwi Akontombra,District,82467,109868
258,Sefwi Wiawso Municipal,Municipal District,139200,185826


<h1><b>LXML</b></h1>

<h2><b>REGIONAL DATA</b></h2>

In [212]:
#scrape using xpath(get this from inspecting the page and copying the full xpath
regions=[] 
cen_pop=[]
proj_pop=[]
area_size=[]

r_i=1
while r_i < 32:
    
    region=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[1]/a/span/text()')
    census=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[3]/text()')
    projected_pop=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[4]/text()')
    area=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr[2]/td/p[4]/span/text()')
    

    
    regions.append(region[0])
    cen_pop.append(census[0])
    proj_pop.append(projected_pop[0])
    area_size.append(area)
    r_i+=2

print(regions)

['Ahafo', 'Ashanti', 'Bono', 'Bono East', 'Central', 'Eastern', 'Greater Accra', 'North East', 'Northern', 'Oti', 'Savannah', 'Upper East', 'Upper West', 'Volta', 'Western', 'Western North']


In [213]:
#create dataframe from data

reg_pop = pd.DataFrame({'Region':regions, 'Census':cen_pop, 'Projected':proj_pop})
reg_pop=reg_pop.replace(',','', regex=True)
reg_pop[['Census', 'Projected']]=reg_pop[['Census', 'Projected']].astype('int64')
print(reg_pop)

#reg_pop.to_csv('Population_by_Region_(GH).csv', index=False)

           Region   Census  Projected
0           Ahafo   484210     613049
1         Ashanti  4780380    5924498
2            Bono   922617    1168807
3       Bono East   904156    1133768
4         Central  2201863    2605492
5         Eastern  2633154    3318853
6   Greater Accra  4010054    5055883
7      North East   465005     588800
8        Northern  1544946    1948913
9             Oti   605725     759799
10       Savannah   469510     594712
11     Upper East  1046545    1302718
12     Upper West   702110     868479
13          Volta  1512527    1907679
14        Western  1664586    2214660
15  Western North   711435     949094


In [7]:
#transform the data using melt so both populations can do a grouped graph

#newreg_pop = pd.melt(reg_pop, id_vars='Region', var_name='Type', value_name='Population')

<h2><b>DISTRICTS DATA</b></h2>

In [178]:
districts=[]
dist_cen_pop=[]
dist_proj_pop=[]
status=[]

In [209]:
districts=[]
dist_cen_pop=[]
dist_proj_pop=[]
status=[]

dbi=2
while dbi<33:
    
    district=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td/a/span/text()')
    census=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[3]/text()')
    proj_pop=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[4]/text()')
    dist_type=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[2]/text()')
    
    for dist in district:
        districts.append(dist)
    for popn1 in census:
        dist_cen_pop.append(popn1)
    for popn2 in proj_pop:
        dist_proj_pop.append(popn2)
    for type in dist_type:
        status.append(type)
    dbi+=2

In [215]:
dist_pop=pd.DataFrame(data={'District':districts, 'Status':status, 'Census':dist_cen_pop, 'Projected':dist_proj_pop})
dist_pop.replace(',','',regex=True,inplace=True)
dist_pop[['Census', 'Projected']]=dist_pop[['Census','Projected']].astype('int64')
print(dist_pop)
#dist_pop.to_csv('Population_by_District_(GH).csv', index=False)

                    District              Status  Census  Projected
0    Asunafo North Municipal  Municipal District  124685     157870
1              Asunafo South            District   95580     120976
2              Asutifi North            District   52259      66141
3              Asutifi South            District   53584      67763
4       Tano North Municipal  Municipal District   79973     101305
..                       ...                 ...     ...        ...
255                     Bodi            District   53314      71133
256                  Juabeso            District   58435      77963
257         Sefwi Akontombra            District   82467     109868
258   Sefwi Wiawso Municipal  Municipal District  139200     185826
259                   Suaman            District   20529      27370

[260 rows x 4 columns]
